In [2]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o", temperature=0)

In [3]:
import requests
from bs4 import BeautifulSoup
from langchain.agents import tool

import csv
import random
from faker import Faker

# Faker 객체 생성 및 한국어 설정
fake = Faker('ko_KR')

# 50명의 이름과 전화번호 생성 및 CSV 파일 저장
def generate_phone_book():
    phone_book = []
    
    for _ in range(50):
        name = fake.name()
        phone = f"010-{random.randint(1000,9999)}-{random.randint(1000,9999)}"
        phone_book.append([name, phone])
    
    with open('phone.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['이름', '전화번호'])
        writer.writerows(phone_book)

# 1. tool을 만든다
# 전화번호로 이름 찾기
@tool
def phone2name(phone_number: str) -> str:
    """
    전화번호로 이름을 찾습니다.

    Args:
        phone_number (str): 찾고자 하는 사람의 전화번호

    Returns:
        str: 해당 전화번호를 가진 사람의 이름 또는 오류 메시지
    """
    with open('phone.csv', 'r', encoding='utf-8') as f:
        reader = csv.reader(f)
        next(reader)  # 헤더 건너뛰기
        for row in reader:
            if row[1] == phone_number:
                return row[0]
    return "해당 전화번호를 가진 사람을 찾을 수 없습니다."

@tool
def name2phone(name: str) -> str:
    """
    이름으로 전화번호를 찾습니다.

    Args:
        name (str): 찾고자 하는 사람의 이름

    Returns:
        str: 해당 이름을 가진 사람의 전화번호 또는 오류 메시지
    """
    with open('phone.csv', 'r', encoding='utf-8') as f:
        reader = csv.reader(f)
        next(reader)  # 헤더 건너뛰기
        for row in reader:
            if row[0] == name:
                return row[1]
    return "해당 이름을 가진 사람을 찾을 수 없습니다."

@tool
def naver_news_crawl(news_url: str) -> str:
    """Crawls a 네비어(naver.com) news article and returns the body content."""
    # HTTP GET 요청 보내기
    response = requests.get(news_url)

    # 요청이 성공했는지 확인
    if response.status_code == 200:
        # BeautifulSoup을 사용하여 HTML 파싱
        soup = BeautifulSoup(response.text, "html.parser")

        # 원하는 정보 추출
        title = soup.find("h2", id="title_area").get_text()
        content = soup.find("div", id="contents").get_text()

        # 출력
        print(f"기사 제목: {title}")
        print(f"본문 내용:\n{content}")
    else:
        print(f"HTTP 요청 실패. 응답 코드: {response.status_code}")
    return content

# 2. tools 를 만든다
tools = [phone2name, name2phone, naver_news_crawl]
# 전화번호부 생성
generate_phone_book()



In [5]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain_community.tools.convert_to_openai import format_tool_to_openai_function
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.agents import AgentExecutor

# 3. 프롬프트를 만든다
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "너는 충실하게 사람들의 요청에 잘 응답해주는 도우미야. 친절하게 답을 해주지만 모르는 것은 모든다고 답해줘야 해.",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

# 4. tools를 활용할 LLM을 만든다
llm_with_tools = llm.bind(functions=[convert_to_openai_function(t) for t in tools])

# 5. tools, LLM, 프롬프트를 결합하는 agent를 만든다
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIFunctionsAgentOutputParser()
)

# 6. agent를 실행하는 executor를 만든다
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [6]:
agent_executor.invoke({"input": "010-2768-4401가 누구지."})["output"]



> Entering new AgentExecutor chain...

Invoking: `phone2name` with `{'phone_number': '010-2768-4401'}`


하재호010-2768-4401은 하재호님의 전화번호입니다.

> Finished chain.


'010-2768-4401은 하재호님의 전화번호입니다.'

In [7]:
agent_executor.invoke(
    {
        "input": "강재현씨 전화번호를 알려주고, 뉴스 기사를 요약해 줘: https://n.news.naver.com/mnews/article/001/0014813208?rc=N&ntype=RANKING"
    }
)["output"]



> Entering new AgentExecutor chain...

Invoking: `name2phone` with `{'name': '강재현'}`


010-2333-2170
Invoking: `naver_news_crawl` with `{'news_url': 'https://n.news.naver.com/mnews/article/001/0014813208?rc=N&ntype=RANKING'}`


기사 제목: 체코원전 '2+α'기 사실상 수주…15년만 '바라카 신화' 재연(종합2보)
본문 내용:



'프랑스 아성' 유럽 원전 첫 진출 눈앞…'온 타임 위딘 버짓' 앞세워 교두보 확보확정된 2기만 24조원 예상…한전그룹사·두산에너빌리티·대우건설 '팀코리아' 참여발주사와 협상 거쳐 내년 3월까지 최종 계약 체결



체코의 신규 원전 예정부지인 두코바니 전경[대우건설 제공. 재판매 및 DB 금지](베를린·세종·서울=연합뉴스) 김계연 특파원 차대운 김동규 이슬기 김영신 기자 = 한국수력원자력이 주축이 된 '팀코리아'가 24조원대로 추산되는 체코 신규 원전 2기 건설 사업의 우선협상대상자로 선정됐다.    이변이 없는 한 한국은 2009년 아랍에미리트(UAE) 바라카 원전 이후 15년 만에 역대 두 번째 원전 수출을 앞두게 됐다.    특히 선진 시장인 유럽에 첫 교두보를 확보해 향후 한국 원전 수출 확대의 중요 계기가 될 수 있다는 기대감도 형성되고 있다.    외신과 산업통상자원부에 따르면 체코 정부는 17일(현지시간) 각료회의를 열고 한수원을 자국 신규 원전 건설 사업의 우선협상대상자로 선정했다고 공식 발표했다.    이로써 한수원은 최종 계약 체결을 위해 발주사인 체코전력공사(CEZ)의 자회사인 두코바니Ⅱ 원자력발전사(EDUⅡ)와 단독으로 협상할 지위를 확보했다.    우선협상대상자 지위를 획득한 한수원은 발주사와 세부 협상을 거쳐 2025년 3월까지 최종 계약을 체결할 계획이다.    체코는 두코바니와 테멜린 지역 원전 단지에 각각 2기씩, 총 4

"강재현씨의 전화번호는 010-2333-2170입니다.\n\n다음은 요청하신 뉴스 기사 요약입니다:\n\n한국수력원자력이 주축이 된 '팀코리아'가 체코 신규 원전 2기 건설 사업의 우선협상대상자로 선정되었습니다. 이 사업은 약 24조원 규모로, 한국은 2009년 UAE 바라카 원전 이후 15년 만에 두 번째 원전 수출을 앞두고 있습니다. 체코 정부는 두코바니 지역에 2기의 원전을 건설할 계획이며, 한수원은 최종 계약을 위해 체코전력공사와 협상을 진행할 예정입니다. 이번 수주는 유럽 원전 시장 진출의 중요한 계기가 될 것으로 기대됩니다."

In [24]:
agent_executor.invoke({"input": "김은숙씨 전화번호 알려줘."})["output"]



> Entering new AgentExecutor chain...

Invoking: `name2phone` with `{'name': '김은숙'}`


해당 이름을 가진 사람을 찾을 수 없습니다.죄송합니다, "김은숙"씨의 전화번호를 찾을 수 없습니다. 다른 이름이나 정보를 제공해 주시면 다시 확인해 보겠습니다.

> Finished chain.


'죄송합니다, "김은숙"씨의 전화번호를 찾을 수 없습니다. 다른 이름이나 정보를 제공해 주시면 다시 확인해 보겠습니다.'